bing 
Request URL: https://cn.bing.com/ttranslatev3?isVertical=1&&IG=A45D80446A634A558A94AE850C68E671&IID=translator.5028.1
Request Method: POST
Status Code: 200 
Remote Address: 202.89.233.101:443
Referrer Policy: origin-when-cross-origin


quest URL: https://cn.bing.com/ttranslatev3?isVertical=1&&IG=A45D80446A634A558A94AE850C68E671&IID=translator.5028.1
Request Method: POST
Status Code: 200 
Remote Address: 202.89.233.101:443
Referrer Policy: origin-when-cross-origin


FROM DATA
fromLang: auto-detect
text: dhu
to: zh-Hans

In [83]:
# -*- encoding=UTF-8
import re, json, random
import requests
from goose3 import Goose
from goose3.text import StopWordsChinese #导入停用词
from bs4 import BeautifulSoup

### 参考
[注释风格](https://zh-google-styleguide.readthedocs.io/en/latest/google-python-styleguide/python_style_rules/)
[翻译api](https://juejin.im/post/5beaac9cf265da614a3a09a9)
[goose提取正文](https://www.ctolib.com/topics-135508.html)
[入门笔记](https://zhuanlan.zhihu.com/c_1231299003028840448)

可修范围：

若干页搜索的解决（我没解决了）

使用更好的选择方式

使用deepl翻译 (这个翻译翻译出来不别扭,nan,尝试失败)

目标字数

混淆程度  （增加互译次数和翻译软件数，）

设置爬虫频率

更换成baidu （bing普适应该更好，好像不是很需要）

添加代理ip轮换？ （爬取频率应该不高，好像不是很需要）


In [109]:

class Ciba:
    '''使用金山词霸翻译
        
    Attributes:
        url: 金山词霸post方法的url
        headers: 报头参数


    '''
    def __init__(self, word, from_lang='auto', to_lang='auto'):
        self.word = word
        self.url = 'http://fy.iciba.com/ajax.php?a=fy'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }
        # 构造post请求的参数
        self.payload = {
            'f': from_lang,
            't': to_lang,
            'w': self.word
        }

    # 发送请求
    def request_post(self):
        try:
            res = requests.post(url=self.url, headers=self.headers, data=self.payload)
            res.raise_for_status()
            # print(res.content.decode())
            return res.content.decode()
        except Exception as e:
            print(e)
            return None
    # 解析数据
    @staticmethod
    def parse_data(data):
        dict_data = json.loads(data)
        if 'out' in dict_data['content']:   #文字段翻译
            return dict_data['content']['out']
        elif 'word_mean' in dict_data['content']:  #词语翻译
            return dict_data['content']['word_mean']

    def run(self):
        data = self.request_post()
        return self.parse_data(data)


In [31]:
def get_access_result(target_url="https://cn.bing.com/", search_word=None):
    """
        Args:
            str:输入搜索字符串

        Returns:
            res:requests返回的response对象

        Raises:
        statuserror:爬取失败

    """
    args={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    url=target_url
    search={}
    if str is not None:
        search['q']=search_word
    try:
        res=requests.get(url, headers=args, params=search)
        res.raise_for_status()
        res.encoding=res.apparent_encoding
        # print(res.text[:1000])
        return res
    except Exception as e:
        print(e)
        print("spider failed")
        return None

In [32]:

def clean_search_result(search_word):
    """
        Args:
            str:输入搜索字符串

        Returns:
            res:返回一组搜索结果和链接

        Raises:
        statuserror:没有搜索结果
        
    """
    res=get_access_result(search_word=search_word)
    soup=BeautifulSoup(res.text, 'lxml')
    clean_res=[]
    for i in soup.findAll(name=['a'], target='_blank', text=re.compile('[^帮助]')):
        # print(i.string,'\n',i.attrs['href'])
        clean_res.append((i.string, i.attrs['href']))
    return clean_res

In [36]:
search_word='抗疫感想'
search_res=clean_search_result(search_word=search_word)

In [34]:
print(repr(search_res))

[('抗疫心得_抗击疫情心得体会范文6篇-理财-钱来也', 'https://www.qianlaiye.com/news/99718.html'), ('抗疫心得体会范文大全【10篇】', 'https://m.51test.net/show/9848999.html'), ('2020普通党员抗疫心得体会范文5篇', 'https://www.chinactp.org/wen/fanwen/392184.html'), ('抗疫心得体会1000字精选三篇', 'http://www.lizhigushi.com/renshengganwu/a21428.html'), ('抗疫志愿者的心得体会 - 心得体会 - 复来作文网', 'http://www.fulay.cn/html/105-176/176319.htm'), ('抗疫心得体会', 'https://www.jxscct.com/fw/52724/'), ('最新党员抗击疫情工作个人感悟_2020战疫工作心得体会_学习力', 'http://www.xuexili.com/xindetihui/3254.html'), ('讲抗疫故事 做奋进青年心得体会范文五篇-钱来也', 'https://www.qianlaiye.com/news/6177043061M2020.html'), ('抗疫作文_感悟英雄在身边', 'https://www.chinactp.org/wen/fanwen/408095.html'), ('新冠肺炎疫情防控心得1000字大全5篇_学习力', 'http://www.xuexili.com/xindetihui/3169.html')]


In [98]:
def get_paragrams(search_res):
    paras=[]
    goose = Goose({'stopwords_class': StopWordsChinese})
    for  ind, res_elem in enumerate(search_res):
        res_herf=res_elem[1]
        article=goose.extract(url=res_herf)
        paras.extend(list(article.cleaned_text.split()))
    return paras

In [103]:
paragrams=get_paragrams(search_res)
# for i in :
#     if not (i =='\n' or i==''):
#          paragrams.append(i)
total=len(paragrams)
print(random.sample(range(total),5))

[151, 144, 162, 187, 74]


In [106]:
total=len(paragrams)
article_gene=''
for i in random.sample(range(total),5):  #生成不重复的随机数
    # print(i, '\n' , paragrams[i])
    article_gene+=paragrams[i]+'\n'
print(article_gene)

另一方面，蜗居在家日久，即使“解封”，人的心理状态也还容易陷入“沉寂”的惯性中，一些人也许会有低沉不振、怀疑过度或无所适从等负面情绪。城市“解封”了，心理也应“解封”。在做好防护的前提下，不妨多看看窗外繁花竞开、草长莺飞的春意，调整心态、提振精神，更积极地投入到学习、工作和生活中。心怀美好希冀，过好接下来的每一天，也是对此前一切牺牲和努力的珍惜。
疫情防控阻击战开始后，监督员不惧风险，冲锋一线，亲临现场，巡回督查。前期指导监督单位选址、设区、分区、配置；中期督促其改建、整改和具体实施，提出合理化建议，下达卫生监督意见书；后期严查被监督单位对疫情防控各项措施的具体落实情况和运行情况，最大限度减少新冠病毒在本场所内的传播风险，真正发挥了卫生监督在疫情防控工作中的重要作用。
“靠前”站位，旗帜定向聚力偕作。紧紧依靠人民群众，把人民群众生命安全和身体健康放在第一位。要在“年关”攻克“难关”，紧急进入应急战斗状态，发挥“集中力量办大事”显著优势，包保到人、执行到底，全力以赴做好各项防控摸排、咨询、宣传工作，不搞对立、不激进，不传谣、不信谣，与鄂同心、稳定民心，既防止“病毒”侵害，又有效遏制焦虑和恐慌蔓延，做好直通身体和心灵的“后半篇文章”。广大党员干部要“靠前”站位，扛起“党员先锋”旗帜，立下战“疫”“请战书”，既当好“战斗员”服从一切调度“召必回”，又当好“指挥员”为科学有效防控出谋划策，保证“战必胜”，特别是农村地区要把握运用好宣传“大喇叭”，领导干部深耕细耕“责任田”，及时应对，守土有责、守土担责、守土尽责。也要把握分寸，我们要抗击的是“病毒”，而不是我们的同胞。这场战“疫”，我们，一起加油。
致敬每一位奋战在防疫前线的90后。谁不知道武汉是疫情最前线?但是，第二批驰援消息刚发下去，20岁的陈琳琳第一个就报了名;推迟了婚礼的申爱、刚订婚不久的尹艳、刚领完结婚证来不及举办婚礼的李婉贞、刚结婚20多天的黄艳清也毅然请战湖北;22岁的漂亮护士胡佩双手被消毒水泡得血痕累累……“2003年，我还太小，17年后刚刚好。”在这场来势汹涌的疫情面前，“逆行”的90后，不怕苦不怕累，以小我成大我，彰显了青年一代的责任与担当。
组建队伍时“我报名”。疫情就是命令，防疫就是责任。面对疫情的蔓延，各地党组织迅速建立起抗击新冠的志愿服务队。号召一经发出，党员们纷纷响应。“我报名”成了彼时的“刷屏

In [110]:
zh_to_en=Ciba(article_gene, 'zh', 'en')
article_en=zh_to_en.run()
en_to_zh=Ciba(article_en, 'en', 'zh')
article_final=en_to_zh.run()
print(article_final)

另一方面，长期生活在家中，即使“启封”，人们的心理状态也容易陷入“沉默”的惯性，有些人可能有低抑郁、过度怀疑或困惑等负面情绪。 市“启封”，心理也应“启封”.. 在保护好的前提下，不妨多看窗外的花争艳，草映飞春，调整心态，提振精神，更积极地投入到学习，工作和生活中.. 这也是所有以前牺牲和努力的财富，有一个美好的希望和美好的一天。
 爆发防控攻坚战开始后，监管人员不畏风险，冲锋一线，亲临现场，巡回监管.. 前期督促监理单位选，分，配；中期督促重建，整改落实，提出合理化建议，出具卫生监督意见；后期严格检查疫情防控措施的具体落实和运行情况，最大限度降低现场新冠状病毒传播风险，真正发挥疫情防控的重要作用..
 “勇往直前”，旗帜定向凝聚在一起。 紧紧依靠人民群众，把人民群众的生命安全和身体健康放在第一位..要克服“新年关口”中的“困难”，紧急进入应急状态，充分发挥“聚精会神做大事”的显著优势，保障人，贯彻到底，全力以赴做好防控，咨询和宣传工作，不搞反对，激进主义，不造谣，不信谣，与湖北同心同德，稳定民心，防止“病毒”侵略，同时有效遏制恐慌蔓延通过身心做好“下半年文章”。 广大党员干部要“挺进”，扛起“党员先锋”的旗帜，为“疫病”树立“战争请愿”。 我们也要把握措施，我们是打“病毒”，不是我们同胞.. 这场战争“流行病”，我们，一起加油..
 向在前线战斗的每一位90后致敬.. 谁不知道武汉是疫情的前线？ 然而，第二批消息刚刚传下去，20岁的陈琳琳第一次报了名；推迟了婚礼的沈爱，刚订婚不久的尹燕，刚办完结婚证来不及举行婚礼的李万珍，刚结婚20多天的黄延庆也坚决地邀请去打湖北；22岁的漂亮水泡护士胡培手上被消毒了血痕。 “2003年，我太年轻了，仅仅17年后。 面对这种疫情，“逆行”90后，不怕苦，不怕累，以自我为大自我，展现了年轻一代的责任和担当。
 当我组建团队的时候。 疫情是秩序，防疫是责任..面对疫情的蔓延，各地党组织迅速组建志愿服务队，抗击新冠.. 号召一经发出，党员响应.. “我报名”成了“必要的刷屏”。 人来自不同的岗位，但只有一个目标，就是团结一心，严防死守，夺取疫情的最终胜利.. 每个人都对世界的兴衰负责。 当人们的生命和财产受到威胁时，共产党人永远不会忽视它们。 积极行动，做到隐患无隐形，彻底清除病毒.. 也许你我的实力是非常有限的，但九千万以上的你我，在

In [ ]:

if __name__ == '__main__':
    word = '突如其来的新型冠状病毒感染的肺炎疫情，既是一场巨大的灾难，也是考察、检验、锻炼基层党组织的一次机遇。广大党员干部要恪尽职守，坚守岗位，做疫情防控的组织员、宣传员、监督员、协调员和战斗员，筑劳疫情防控的坚强“红色防线”。各级党委要敏锐地认识和把握住契机，在“疫考”中加紧锤炼基层党组织，为打赢疫情防控阻击战提供坚强的组织保证。'
    Ciba = Ciba(word,'zh','en')
    print(Ciba.run())


# 作废
def get_translation_result(str, source_lang, target_lang):
    """
        Args:
            str:输入搜索字符串

        Returns:
            res:requests返回的response对象

        Raises:
        statuserror:爬取失败

    """
    args={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    url="https://www.deepl.com/translator"+'#'+source_lang+'/'+target_lang+'/'+str
    print(url)

    try:
        res=requests.get(url, headers=args, params=search)
        res.raise_for_status()
        res.encoding=res.apparent_encoding
        # print(res.text[:1000])
        soup=BeautifulSoup(res.text, 'html.parser')
        trans_tag=soup.find(name='button', attrs={'class':'lmt__translations_as_text__text_btn'})
        print(trans_tag.string)
    except:
        print("spider failed", res.requests.header.status)
    

# 作废
word='巴拉巴拉'
get_translation_result(word, 'zh', 'en')

# 作废
args={'authority': 'www2.deepl.com',
    'method': 'POST',
    'path': '/jsonrpc',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6',
    'content-length': '381',
    'content-type': 'text/plain',
    'cookie': 'LMTBID=4592ab4d-d12e-4e06-bd8b-fac83261bb7c|             45d64d2bad21f9240f08bb7f8e141e64; _ga=GA1.2.472039245.1588292717; _gid=GA1.2.521171080.1588292717',
    'dnt': '1',
    'origin': 'https://www.deepl.com',
    'referer': 'https://www.deepl.com/translator',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'}
payload={
        'jsonrpc': '2.0',
        'method': 'LMT_handle_jobs',
        'params': {
            'jobs': [
                {
                    'kind':'default',
                    'raw_en_sentence': '好好地'
                }
            ],
            'lang': {
                'user_preferred_langs': [
                    'en',
                    'zh'
                ],
                'source_lang_user_selected': 'zh',
                'target_lang': 'en'
            },
            'priority': -1
        }
}
res=requests.post('https://www2.deepl.com/jsonrpc', headers=args, data=json.dumps(payload))
print(res.status_code)
print(repr(json.loads(res.content)))
# trans_tag=soup.find(name='div', attrs={'class':'lmt__textarea_base_style'})
# for child in trans_tag.descendants:
#     print(child.string)